In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from collections import Counter
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from nltk import bigrams, pos_tag




In [ ]:
india_labelled=pd.read_csv('/kaggle/input/final-project/india_gpt_labelled.csv')

In [ ]:
america_labelled=pd.read_csv('/kaggle/input/final-project/America_final_labelled.csv')

In [ ]:
canada_labelled=pd.read_csv('/kaggle/input/final-project/canada_gpt_labelled.csv')

In [ ]:
canada_labelled

In [ ]:
india_labelled

In [ ]:
canada_labelled.manual_labels.value_counts()

In [ ]:
askindia_labelled=pd.read_csv('/kaggle/input/final-project/AskIndia_labelled_final_v2.csv')

In [ ]:
askindia_labelled

In [ ]:
askindia_labelled.results.value_counts()

In [ ]:
df = pd.concat([india_labelled], ignore_index=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df=df.dropna()

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [ ]:
cleaned_comments=[]
for c in tqdm(df['comments'],total=len(df['comments']),desc='Cleaning'):
    og_comment=c
#     try:
#         c=remove_html_tags(c)
#         c=handle_urls_and_mentions(c)
#         c=remove_special_characters(c)
#         c=convert_to_lowercase(c)
    c=remove_stopwords(c)
        #     c=lemmatize_text(c)
#         c=lemmatize_text_spacy(c)
#     except Exception as e:
#         print(f"Error extracting keywords for {og_comment} : {e}")
#         continue
        
    cleaned_comments.append(c)

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(cleaned_comments, df['manual_labels'], test_size=0.2, random_state=69)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
train_vectors = vectorizer.fit_transform(train_data)
test_vectors = vectorizer.transform(test_data)

In [ ]:
classifier = MultinomialNB()
classifier.fit(train_vectors, train_labels)

In [ ]:
predictions = classifier.predict(test_vectors)

In [ ]:
print("Accuracy:", accuracy_score(test_labels, predictions))
print("\nClassification Report:\n", classification_report(test_labels, predictions))

In [ ]:
unique, counts = np.unique(predictions, return_counts=True)
value_counts = dict(zip(unique, counts))
print(value_counts)

In [ ]:
test_labels.value_counts()

In [ ]:
pred_df = pd.DataFrame({'pred': predictions, 'label': test_labels})
display(pred_df.sample(5, random_state=69))

In [ ]:
pred_df['correct'] = pred_df.label == pred_df.pred
# YOUR CODE HERE
class_size = pred_df.groupby('label').size()
class_accuracy = pred_df.groupby('label')['correct'].mean() * 100
class_size_dict = class_size.to_dict()
class_accuracy_dict = class_accuracy.to_dict()
print("Class Size:", class_size_dict)
print("Class Accuracy:", class_accuracy_dict)

In [ ]:
log=LogisticRegression(multi_class='multinomial')
log.fit(train_vectors, train_labels)

In [ ]:
predictions = log.predict(test_vectors)

In [ ]:
print("Accuracy:", accuracy_score(test_labels, predictions))
print("\nClassification Report:\n", classification_report(test_labels, predictions))

In [ ]:
unique, counts = np.unique(predictions, return_counts=True)
value_counts = dict(zip(unique, counts))
print(value_counts)

In [ ]:
coefs = log.coef_
features = vectorizer.get_feature_names_out()
classes = log.classes_
coef_df = pd.DataFrame(data=coefs.T, index=features, columns=classes)
display(coef_df)
#most predictive words
print(coef_df[-1].sort_values(ascending=False).head(10))
print(coef_df[1].sort_values(ascending=False).head(10))

In [ ]:
pred_df = pd.DataFrame({'pred': predictions, 'label': test_labels})
display(pred_df.sample(5, random_state=69))

In [ ]:
pred_df['correct'] = pred_df.label == pred_df.pred
# YOUR CODE HERE
class_size = pred_df.groupby('label').size()
class_accuracy = pred_df.groupby('label')['correct'].mean() * 100
class_size_dict = class_size.to_dict()
class_accuracy_dict = class_accuracy.to_dict()
print("Class Size:", class_size_dict)
print("Class Accuracy:", class_accuracy_dict)

In [ ]:
df['xgboost_preds']=predictions

In [ ]:
train_labels+1

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

xgb_model = xgb.XGBClassifier()

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)

# Fit the model to the data
grid_search.fit(train_vectors, train_labels+1)


In [ ]:
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)

In [ ]:
predictions = grid_search.predict(test_vectors)

In [ ]:
print("Accuracy:", accuracy_score(test_labels+1, predictions))
print("\nClassification Report:\n", classification_report(test_labels+1, predictions))

In [ ]:
test_labels+1

In [ ]:
pred_df = pd.DataFrame({'pred': predictions, 'label': test_labels+1})
display(pred_df.sample(5, random_state=69))

In [ ]:
unique, counts = np.unique(predictions, return_counts=True)
value_counts = dict(zip(unique, counts))
print(value_counts)

In [ ]:
pred_df['correct'] = pred_df.label == pred_df.pred
# YOUR CODE HERE
class_size = pred_df.groupby('label').size()
class_accuracy = pred_df.groupby('label')['correct'].mean() * 100
class_size_dict = class_size.to_dict()
class_accuracy_dict = class_accuracy.to_dict()
print("Class Size:", class_size_dict)
print("Class Accuracy:", class_accuracy_dict)

In [ ]:
all_words = [word for comment in train_data for word in word_tokenize(comment)]
vocab_size = len(Counter(all_words))

In [ ]:
max_words = vocab_size
max_len = 500
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(train_data)
train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)
train_padded = pad_sequences(train_sequences, maxlen=max_len)
test_padded = pad_sequences(test_sequences, maxlen=max_len)

In [ ]:
train_padded.shape

In [ ]:
model = Sequential()
model.add(Embedding(max_words,  input_length=max_len, trainable=False,output_dim=240))
# model.add(SpatialDropout1D(0.2))
# model.add(LSTM(100, return_sequences=True))  # Use return_sequences=True for stacking LSTM layers
# model.add(LSTM(100, kernel_regularizer=l2(0.01)))
# model.add(Dense(128, activation='relu'))  
# # model.add(Dense(64, activation='relu'))   
# model.add(Dropout(0.5))


model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 16 , activation = 'relu'))
model.add(Dense(3, activation='softmax'))
# model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_padded, train_labels+1, epochs=50, batch_size=8, validation_split=0.2)